# 1. Imports

In [1]:
import pandas as pd
import numpy as np
import pymove
from pymove import folium, MoveDataFrame
from pymove.query import query
from datetime import datetime
from numpy import Inf

# 2. Load Data

DataSet
-   [Hurricanes and Typhoons](https://www.kaggle.com/noaa/hurricane-database):he NHC publishes the tropical cyclone historical database in a format known as HURDAT, short for HURricane DATabase

In [2]:
hurricanes_pandas_df = pd.read_csv('atlantic.csv')
hurricanes_pandas_df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,HU,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,HU,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,,EX,41.3N,50.4W,55,981,...,180,120,120,120,60,0,0,0,0,0
49101,AL122015,KATE,20151112,1800,,EX,41.9N,49.9W,55,983,...,180,120,120,120,60,0,0,0,0,0
49102,AL122015,KATE,20151113,0,,EX,41.5N,49.2W,50,985,...,200,220,120,120,60,0,0,0,0,0
49103,AL122015,KATE,20151113,600,,EX,40.8N,47.5W,45,985,...,180,220,0,0,0,0,0,0,0,0


In [3]:
#Select hurricanes from 2012 to 2015
hurricanes_pandas_df = hurricanes_pandas_df.loc[hurricanes_pandas_df['Date'] >= 20120000]
hurricanes_pandas_df = hurricanes_pandas_df.loc[hurricanes_pandas_df['Date'] < 20160000]
hurricanes_pandas_df.shape

(1639, 22)

In [4]:
#Converting N, S, W, L format to integer latitude and longitude
copy_lat = hurricanes_pandas_df['Latitude'].copy()
copy_lon = hurricanes_pandas_df['Longitude'].copy()
for i in hurricanes_pandas_df.index:
    if (hurricanes_pandas_df['Latitude'][i][-1:] == 'N'):
        hurricanes_pandas_df['Latitude'][i] = float(copy_lat[i][:-1])
    else: 
        hurricanes_pandas_df['Latitude'][i] = float(copy_lat[i][:-1]) * -1
    if (hurricanes_pandas_df['Longitude'][i][-1:] == 'E'):
        hurricanes_pandas_df['Longitude'][i] = float(copy_lat[i][:-1]) 
    else:
        hurricanes_pandas_df['Longitude'][i] = float(copy_lon[i][:-1]) * -1
hurricanes_pandas_df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
47466,AL012012,ALBERTO,20120519,0,,LO,33.1,-77,25,1010,...,0,0,0,0,0,0,0,0,0,0
47467,AL012012,ALBERTO,20120519,600,,TD,32.8,-77.1,30,1008,...,0,0,0,0,0,0,0,0,0,0
47468,AL012012,ALBERTO,20120519,1200,,TS,32.5,-77.3,40,1005,...,30,30,0,0,0,0,0,0,0,0
47469,AL012012,ALBERTO,20120519,1800,,TS,32.3,-77.6,45,997,...,30,40,0,0,0,0,0,0,0,0
47470,AL012012,ALBERTO,20120520,0,,TS,32.1,-78.1,50,995,...,30,40,0,0,0,20,0,0,0,0


In [5]:
#Creating the DateTime column by merging the Time column and the Date column
hurricanes_pandas_df['datetime'] = hurricanes_pandas_df['Time'].copy()
for i in hurricanes_pandas_df.index:
    this_date = str(hurricanes_pandas_df['Date'][i])[0:4] + '-' + str(hurricanes_pandas_df['Date'][i])[4:6] + '-' + str(hurricanes_pandas_df['Date'][i])[6:]
    this_time = ' ' + str(int(hurricanes_pandas_df['Time'][i]/100)) + ':' + str(hurricanes_pandas_df['Time'][i])[-2:] + ':00'
    hurricanes_pandas_df['datetime'][i] = this_date + this_time
hurricanes_pandas_df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,datetime
47466,AL012012,ALBERTO,20120519,0,,LO,33.1,-77,25,1010,...,0,0,0,0,0,0,0,0,0,2012-05-19 0:0:00
47467,AL012012,ALBERTO,20120519,600,,TD,32.8,-77.1,30,1008,...,0,0,0,0,0,0,0,0,0,2012-05-19 6:00:00
47468,AL012012,ALBERTO,20120519,1200,,TS,32.5,-77.3,40,1005,...,30,0,0,0,0,0,0,0,0,2012-05-19 12:00:00
47469,AL012012,ALBERTO,20120519,1800,,TS,32.3,-77.6,45,997,...,40,0,0,0,0,0,0,0,0,2012-05-19 18:00:00
47470,AL012012,ALBERTO,20120520,0,,TS,32.1,-78.1,50,995,...,40,0,0,0,20,0,0,0,0,2012-05-20 0:0:00


In [6]:
#Converting the pandas dataframe to pymove's MoveDataFrame
hurricanes_2012 = MoveDataFrame(
    data=hurricanes_pandas_df, latitude='Latitude', longitude='Longitude',datetime='datetime', traj_id='Name'
)
hurricanes_2012.head()

,ID,id,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,...,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,datetime
47466,AL012012,ALBERTO,20120519,0,,LO,33.1,-77.0,25,1010,...,0,0,0,0,0,0,0,0,0,2012-05-19 00:00:00
47467,AL012012,ALBERTO,20120519,600,,TD,32.8,-77.1,30,1008,...,0,0,0,0,0,0,0,0,0,2012-05-19 06:00:00
47468,AL012012,ALBERTO,20120519,1200,,TS,32.5,-77.3,40,1005,...,30,0,0,0,0,0,0,0,0,2012-05-19 12:00:00
47469,AL012012,ALBERTO,20120519,1800,,TS,32.3,-77.6,45,997,...,40,0,0,0,0,0,0,0,0,2012-05-19 18:00:00
47470,AL012012,ALBERTO,20120520,0,,TS,32.1,-78.1,50,995,...,40,0,0,0,20,0,0,0,0,2012-05-20 00:00:00


#### Visualization

In [7]:
folium.plot_trajectories_with_folium(hurricanes_2012, zoom_start=2)

&lt;folium.folium.Map at 0x7fc4d071d710&gt;

In [8]:
#Total hurricane amount between 2012 and 2015
this_ex = hurricanes_2012
this_ex['id'].unique().shape[0]

55

In [9]:
#Selecting a hurricane for demonstration
gonzalo = hurricanes_2012.loc[hurricanes_2012['id'] == '            GONZALO']

In [10]:
folium.plot_trajectories_with_folium(
    gonzalo, lat_origin=gonzalo['lat'].median(), lon_origin=gonzalo['lon'].median(), zoom_start=2
    
)

&lt;folium.folium.Map at 0x7fc4a6300290&gt;

# 2. Range Query

### Using distance MEDP (Mean Euclidean Distance Predictive)

In [11]:
prox_Gonzalo = query.range_query(gonzalo, hurricanes_2012, range=200, distance="MEDP")

In [12]:
folium.plot_trajectories_with_folium(prox_Gonzalo, zoom_start=3)

&lt;folium.folium.Map at 0x7fc4a62dbad0&gt;

### Using Distance MEDT (Mean Euclidean Distance Trajectory)

In [13]:
prox_Gonzalo = query.range_query(gonzalo, hurricanes_2012, range=1000, distance='MEDT')

In [14]:
folium.plot_trajectories_with_folium(prox_Gonzalo, zoom_start=3)

&lt;folium.folium.Map at 0x7fc4a62db350&gt;

# 3. KNN (K-Nearest-Neighbor)

### Using distance MEDP (Mean Euclidean Distance Predictive)

In [15]:
prox_Gonzalo = query.knn_query(gonzalo, hurricanes_2012, id='id', k=5, distance='MEDP')

Gerando DataFrame com as k trajetórias mais próximas


In [16]:
folium.plot_trajectories_with_folium(prox_Gonzalo, zoom_start=3)

&lt;folium.folium.Map at 0x7fc4a62d7b10&gt;

### Using Distance MEDT (Mean Euclidean Distance Trajectory)

In [17]:
prox_Gonzalo = query.knn_query(gonzalo, hurricanes_2012, id='id', k=5, distance='MEDT')

Gerando DataFrame com as k trajetórias mais próximas


In [18]:
folium.plot_trajectories_with_folium(prox_Gonzalo, zoom_start=3)

&lt;folium.folium.Map at 0x7fc4a621dd10&gt;